In [1]:
import math
import mne
import numpy as np 
import pandas as pd  
import pickle
import random 

from os.path import exists 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Set the parameters
recording_time=25
file_name =['h','s']
h_num=14
s_num=14
iir_params = {'order': 2, 'ftype': 'butter'}


In [3]:
# Random Subjects shuffling
indice=list(range(1,h_num+1))
random.shuffle(indice)

print(indice)

[4, 3, 6, 10, 9, 12, 14, 13, 7, 5, 8, 11, 1, 2]


In [4]:
# Selecting Channels 
def select_channel(raw,req_channel=['Fp1','Fp2']):
    indicess=[]
    for channel in req_channel:
        indicess.append(raw.ch_names.index(channel))
    new_raw=raw.get_data(picks=indice)
    return new_raw

In [5]:
#NORMALIZE THE SELECTED DATA
def normalize_eeg_data(new_raw):
    scaler=MinMaxScaler()
    transposed_raw=new_raw.T
    normalized_raw=scaler.fit_transform(transposed_raw)
    new_raw=normalized_raw.T
    return new_raw



In [6]:
def sliding_window_augmentation(data, window_size, stride):
    data=np.array(data)
    n_channels,n_timepoints,  = data.shape
    augmented_data = []
    for start in range(0, n_timepoints - window_size + 1, stride):
        end = start + window_size
        segment = data[:,start:end]
        augmented_data.append(segment)
    return np.array(augmented_data)




In [7]:
# Dividing Into Chunks
def division(new_raw,label,sampling_fre=250,trail_time=30,augment_ratio=2):
    trail_length=sampling_fre*trail_time
    stride=(trail_length//augment_ratio)
    t_l_signal=new_raw.shape[1]
    chunks_amount=t_l_signal//trail_length
    new_raw=new_raw[:,0:chunks_amount*trail_length]
    augmented_eeg_data = sliding_window_augmentation(new_raw, trail_length, stride)
    features=[]
    for i in augmented_eeg_data:
        features.append((i,label))
    return features


In [9]:
# Combining all and preprocessing the whole data
def preprocess(subject_name,indices,labels,number_of_subjects=14,train_percent=0.8,Band_Pass_Low_Range=0,Band_Pass_High_Range=50,trail_times=30,augment_ratio=2):
    limit=math.ceil(train_percent*number_of_subjects)
    k=1
    train_data=[]
    test_data=[]
    for subject_number in indices:
        if exists('./Dataset/{}{}.edf'.format(subject_name,subject_number)):
            file='./Dataset/{}{}.edf'.format(subject_name,subject_number)
            raw=mne.io.read_raw_edf(file,verbose=0,preload=True,eog=None,exclude=(),stim_channel='auto')
            raw=raw.pick_types(eeg=True)
            raw=raw.filter(Band_Pass_Low_Range,Band_Pass_High_Range,iir_params=iir_params,method='iir')
            new_raw=select_channel(raw)
            new_raw=normalize_eeg_data(new_raw)
            chunks=division(new_raw,label=labels,trail_time=trail_times,augment_ratio=augment_ratio)
            if k<=limit:
                train_data+=chunks
            else:
                test_data+=chunks
            k=k+1
    return train_data,test_data



In [10]:
pos_class_train,pos_class_test=preprocess('h',indices=indice,labels=0,trail_times=recording_time,augment_ratio=2)
neg_class_train,neg_class_test=preprocess('s',indices=indice,labels=1,trail_times=recording_time,augment_ratio=2)
testing_data=pos_class_test+neg_class_test

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 4 (effective, after forward-backward)
- Cutoff at 50.00 Hz: -6.02 dB

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 4 (effective, after forward-backward)
- Cutoff at 50.00 Hz: -6.02 dB

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non

In [12]:
# Using double backslashes
with open('./samples/pos_class_train.pkl', 'wb') as f:
    pickle.dump(pos_class_train, f)

with open('./samples/neg_class_train.pkl', 'wb') as f:
    pickle.dump(neg_class_train, f)

with open('./samples/testing_augmented.pkl', 'wb') as f:
    pickle.dump(testing_data, f)